In [83]:
import pandas as pd
train_data = pd.read_csv('./data/full_train_data.csv',)
valid_data = pd.read_csv('./data/full_valid_data.csv')
test_data = pd.read_csv('./data/full_test_data.csv')

In [84]:
# Find the column index with the maximum value for each row
dominant_topic_index = train_data[['topics_val1', 'topics_val2', 'topics_val3', 'topics_val4', 'topics_val5']].idxmax(axis=1)

# Convert column index to categorical number (1 to 5)
train_data['dominant_topics'] = dominant_topic_index.str.extract('(\d+)').astype(int)

In [85]:
# Find the column index with the maximum value for each row
dominant_topic_index = test_data[['topics_val1', 'topics_val2', 'topics_val3', 'topics_val4', 'topics_val5']].idxmax(axis=1)

# Convert column index to categorical number (1 to 5)
test_data['dominant_topics'] = dominant_topic_index.str.extract('(\d+)').astype(int)

In [86]:
train_data.columns

Index(['item_title', 'item_author_cate', 'article_author',
       'article_source_cate', 'month', 'eastmoney_robo_journalism',
       'media_robo_journalism', 'SMA_robo_journalism', 'viral',
       'sentiment_score', 'topics_val1', 'topics_val2', 'topics_val3',
       'topics_val4', 'topics_val5', 'stock_code', 'IndustryCode1',
       'IndustryName1', 'IndustryCode2', 'IndustryName2',
       'item_author_reduced', 'article_author_reduced',
       'article_source_reduced', 'stock_code_index', 'item_author_cate_index',
       'article_author_index', 'article_source_cate_index', 'month_index',
       'IndustryCode1_index', 'IndustryCode2_index',
       'eastmoney_robo_journalism_index', 'media_robo_journalism_index',
       'SMA_robo_journalism_index', 'item_author_reduced_index',
       'article_author_reduced_index', 'article_source_reduced_index',
       'neg_item_title', 'neg_item_author_cate', 'neg_article_author',
       'neg_article_source_cate', 'neg_month', 'neg_eastmoney_robo_jo

In [98]:
input_list = ['month_index', 
            'eastmoney_robo_journalism_index', 
            'media_robo_journalism_index', 
            'SMA_robo_journalism_index',
            'sentiment_score',
            'stock_code_index', 
            'IndustryCode1_index',
            'IndustryCode2_index',
            'dominant_topics',
            'item_author_reduced_index', 
            'article_author_reduced_index',
            'article_source_reduced_index']

# 划分特征和目标变量
train_x = train_data[input_list]
train_y = train_data['viral']
# valid_x = valid_data[input_list]
# valid_y = valid_data['viral']
test_x = test_data[input_list]
test_y = test_data['viral']

In [89]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# cat_col_list = ['month_index', 
#             'eastmoney_robo_journalism_index', 
#             'media_robo_journalism_index', 
#             'SMA_robo_journalism_index',
#             'stock_code_index', 
#             'IndustryCode1_index',
#             'IndustryCode2_index',
#             'dominant_topics',
#             'item_author_reduced_index', 
#             'article_author_reduced_index',
#             'article_source_reduced_index']

train_x['month_index'] = le.fit_transform(train_x['month_index'].astype(int))
train_x['eastmoney_robo_journalism_index'] = le.fit_transform(train_x['eastmoney_robo_journalism_index'].astype(int))
train_x['media_robo_journalism_index'] = le.fit_transform(train_x['media_robo_journalism_index'].astype(int))
train_x['stock_code_index'] = le.fit_transform(train_x['stock_code_index'].astype(int))
train_x['IndustryCode1_index'] = le.fit_transform(train_x['IndustryCode1_index'].astype(int))
train_x['IndustryCode2_index'] = le.fit_transform(train_x['IndustryCode2_index'].astype(int))
train_x['dominant_topics'] = le.fit_transform(train_x['dominant_topics'].astype(int))
train_x['item_author_reduced_index'] = le.fit_transform(train_x['item_author_reduced_index'].astype(int))
train_x['article_author_reduced_index'] = le.fit_transform(train_x['article_author_reduced_index'].astype(int))
train_x['article_source_reduced_index'] = le.fit_transform(train_x['article_source_reduced_index'].astype(int))

test_x['month_index'] = le.fit_transform(test_x['month_index'].astype(int))
test_x['eastmoney_robo_journalism_index'] = le.fit_transform(test_x['eastmoney_robo_journalism_index'].astype(int))
test_x['media_robo_journalism_index'] = le.fit_transform(test_x['media_robo_journalism_index'].astype(int))
test_x['stock_code_index'] = le.fit_transform(test_x['stock_code_index'].astype(int))
test_x['IndustryCode1_index'] = le.fit_transform(test_x['IndustryCode1_index'].astype(int))
test_x['IndustryCode2_index'] = le.fit_transform(test_x['IndustryCode2_index'].astype(int))
test_x['dominant_topics'] = le.fit_transform(test_x['dominant_topics'].astype(int))
test_x['item_author_reduced_index'] = le.fit_transform(test_x['item_author_reduced_index'].astype(int))
test_x['article_author_reduced_index'] = le.fit_transform(test_x['article_author_reduced_index'].astype(int))
test_x['article_source_reduced_index'] = le.fit_transform(test_x['article_source_reduced_index'].astype(int))


# train_x[cat_col_list] = le.fit_transform(train_x[cat_col_list])
train_y = le.fit_transform(train_y)
# test_x[cat_col_list] = le.fit_transform(test_x[cat_col_list])
test_y = le.fit_transform(test_y)

/var/folders/52/5ygfc9d94sscvjzdx8yvt2sm0000gq/T/ipykernel_71286/2778505005.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['month_index'] = le.fit_transform(train_x['month_index'].astype(int))
/var/folders/52/5ygfc9d94sscvjzdx8yvt2sm0000gq/T/ipykernel_71286/2778505005.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['eastmoney_robo_journalism_index'] = le.fit_transform(train_x['eastmoney_robo_journalism_index'].astype(int))
/var/folders/52/5ygfc9d94sscvjzdx8yvt2sm0000gq/T/ipykernel

# Use machine learning models to predict virality

## XGboost

In [62]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ndcg_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV


# 初始化分类器
model1 = xgb.XGBClassifier()

# 参数网格
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.3, 0.5, 0.7]
    }

# 设置GridSearchCV
grid_search = GridSearchCV(model1, param_grid, scoring='accuracy', cv=3, verbose=3)
grid_search.fit(train_x, train_y)

# 使用最佳参数的模型
best_model = grid_search.best_estimator_

# 建立XGBoost模型
# 从GridSearchCV获得最佳参数设置
model1 = xgb.XGBClassifier(
    learning_rate=best_model.learning_rate,
    n_estimators=best_model.n_estimators,
    max_depth= best_model.max_depth,
    subsample=best_model.subsample,
    colsample_bytree=best_model.colsample_bytree
    )
model1.fit(train_x, train_y)

# 预测
test_pred = model1.predict(test_x)

Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV 1/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=1.000 total time=   0.1s
[CV 2/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=1.000 total time=   0.1s
[CV 3/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=1.000 total time=   0.1s
[CV 1/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9;, score=1.000 total time=   0.1s
[CV 2/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9;, score=1.000 total time=   0.1s
[CV 3/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9;, score=1.000 total time=   0.1s
[CV 1/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0;, score=1.000 total time=   0.1s
[CV 2/

KeyboardInterrupt: 

In [16]:
# 评估：accuracy
print("Classification Report:\n", classification_report(test_y, test_pred))
conf_matrix = confusion_matrix(test_y, test_pred)
print("Confusion Matrix:\n", conf_matrix)

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6354
           1       0.00      0.00      0.00        46

    accuracy                           0.99      6400
   macro avg       0.50      0.50      0.50      6400
weighted avg       0.99      0.99      0.99      6400

Confusion Matrix:
 [[6354    0]
 [  46    0]]


/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

In [17]:
# 评估：ndcg
print("NDCG@1:\n", ndcg_score([test_y], [test_pred], k=1))
print("NDCG@5:\n", ndcg_score([test_y], [test_pred], k=5))
print("NDCG@10:\n", ndcg_score([test_y], [test_pred], k=10))
print("NDCG:\n", ndcg_score([test_y], [test_pred]))

NDCG@1:
 0.0071875
NDCG@5:
 0.007187499999999999
NDCG@10:
 0.0071875
NDCG:
 0.3458997499253763


Feature Importance

In [18]:
# 获取特征的重要性
feature_importance = model1.feature_importances_

# 将特征重要性与特征名称进行配对，并排序
feature_importance_dict = dict(zip(test_x.columns, feature_importance))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# 打印特征重要性
print("Feature importances:\n", sorted_feature_importance)

Feature importances:
 [('month_index', 0.0), ('eastmoney_robo_journalism_index', 0.0), ('media_robo_journalism_index', 0.0), ('SMA_robo_journalism_index', 0.0), ('sentiment_score', 0.0), ('stock_code_index', 0.0), ('IndustryCode1_index', 0.0), ('IndustryCode2_index', 0.0), ('dominant_topics', 0.0), ('item_author_reduced_index', 0.0), ('article_author_reduced_index', 0.0), ('article_source_reduced_index', 0.0)]


## Random Forest

Due to network issues, the process of choosing best hyparameters was run in three separate runs.

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


# 参数范围设置
max_depth_options = [10, 20, 30]
n_estimators_options = [50, 80, 100, 150, 200]

best_score = 0
best_params = {'max_depth': None, 'n_estimators': 100}

for max_depth in max_depth_options:
    for n_estimators in n_estimators_options:
        # 初始化随机森林分类器
        model = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, random_state=42)

        # 使用交叉验证计算评分
        scores = cross_val_score(model, train_x, train_y, cv=5, scoring='accuracy')
        average_score = np.mean(scores)

        print(f"Testing {n_estimators} trees with max depth of {max_depth}. Mean CV Accuracy: {average_score}")

        # 检查并更新最佳得分和参数
        if average_score > best_score:
            best_score = average_score
            best_params = {'max_depth': max_depth, 'n_estimators': n_estimators}

Testing 50 trees with max depth of 10. Mean CV Accuracy: 1.0
Testing 80 trees with max depth of 10. Mean CV Accuracy: 1.0
Testing 100 trees with max depth of 10. Mean CV Accuracy: 1.0
Testing 150 trees with max depth of 10. Mean CV Accuracy: 1.0
Testing 200 trees with max depth of 10. Mean CV Accuracy: 1.0
Testing 50 trees with max depth of 20. Mean CV Accuracy: 1.0
Testing 80 trees with max depth of 20. Mean CV Accuracy: 1.0
Testing 100 trees with max depth of 20. Mean CV Accuracy: 1.0
Testing 150 trees with max depth of 20. Mean CV Accuracy: 1.0
Testing 200 trees with max depth of 20. Mean CV Accuracy: 1.0
Testing 50 trees with max depth of 30. Mean CV Accuracy: 1.0
Testing 80 trees with max depth of 30. Mean CV Accuracy: 1.0
Testing 100 trees with max depth of 30. Mean CV Accuracy: 1.0
Testing 150 trees with max depth of 30. Mean CV Accuracy: 1.0
Testing 200 trees with max depth of 30. Mean CV Accuracy: 1.0


In [ ]:
# 使用最佳参数训练模型
best_model = RandomForestClassifier(max_depth=20, n_estimators=80, random_state=42)
best_model.fit(train_x, train_y)
test_pred = best_model.predict(test_x)

# 评估
print("Classification Report:\n", classification_report(test_y, test_pred))
conf_matrix = confusion_matrix(test_y, test_pred)
print("Confusion Matrix:\n", conf_matrix)

# 评估：ndcg
print("NDCG@1:\n", ndcg_score([test_y], [test_pred], k=1))
print("NDCG@5:\n", ndcg_score([test_y], [test_pred], k=5))
print("NDCG@10:\n", ndcg_score([test_y], [test_pred], k=10))
print("NDCG:\n", ndcg_score([test_y], [test_pred]))

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6354
           1       0.00      0.00      0.00        46

    accuracy                           0.99      6400
   macro avg       0.50      0.50      0.50      6400
weighted avg       0.99      0.99      0.99      6400

Confusion Matrix:
 [[6354    0]
 [  46    0]]
NDCG@1:
 0.0071875
NDCG@5:
 0.007187499999999999
NDCG@10:
 0.0071875
NDCG:
 0.3458997499253763


/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

In [23]:
# 获取特征的重要性
feature_importance = best_model.feature_importances_

# 将特征重要性与特征名称进行配对，并排序
feature_importance_dict = dict(zip(test_x, feature_importance))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

print("Feature importances:\n", sorted_feature_importance)

Feature importances:
 [('month_index', 0.0), ('eastmoney_robo_journalism_index', 0.0), ('media_robo_journalism_index', 0.0), ('SMA_robo_journalism_index', 0.0), ('sentiment_score', 0.0), ('stock_code_index', 0.0), ('IndustryCode1_index', 0.0), ('IndustryCode2_index', 0.0), ('dominant_topics', 0.0), ('item_author_reduced_index', 0.0), ('article_author_reduced_index', 0.0), ('article_source_reduced_index', 0.0)]


## Logistic Regression

Add class weights

In [69]:
input_list = ['month_index', 
            'eastmoney_robo_journalism_index', 
            'media_robo_journalism_index', 
            'SMA_robo_journalism_index',
            'stock_code_index', 
            'IndustryCode1_index',
            'IndustryCode2_index',
            'dominant_topics',
            'item_author_reduced_index', 
            'article_author_reduced_index',
            'article_source_reduced_index']

encoder = OneHotEncoder(sparse_output=False)

train_x_onehot, test_x_onehot = train_x, test_x

for col in input_list:
    col_onehot = encoder.fit_transform(train_x_onehot[[col]])
    encoded_df = pd.DataFrame(col_onehot, columns=[f"{col}_{i}" for i in range(col_onehot.shape[1])])
    train_x_onehot = pd.concat([train_x_onehot.drop(col, axis=1), encoded_df], axis=1)

for col in input_list:
    col_onehot = encoder.fit_transform(test_x_onehot[[col]])
    encoded_df = pd.DataFrame(col_onehot, columns=[f"{col}_{i}" for i in range(col_onehot.shape[1])])
    test_x_onehot = pd.concat([test_x_onehot.drop(col, axis=1), encoded_df], axis=1)


In [70]:
train_x_onehot.columns

Index(['sentiment_score', 'month_index_0', 'month_index_1', 'month_index_2',
       'month_index_3', 'month_index_4', 'month_index_5', 'month_index_6',
       'month_index_7', 'month_index_8',
       ...
       'item_author_reduced_index_5', 'item_author_reduced_index_6',
       'item_author_reduced_index_7', 'item_author_reduced_index_8',
       'item_author_reduced_index_9', 'article_author_reduced_index_0',
       'article_source_reduced_index_0', 'article_source_reduced_index_1',
       'article_source_reduced_index_2', 'article_source_reduced_index_3'],
      dtype='object', length=183)

In [104]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 对 dominant_topic 进行独热编码


# 标准化
scaler = StandardScaler()
sentiment_scores = train_x_onehot['sentiment_score'].values.reshape(-1, 1)
train_x_onehot['sentiment_score'] = scaler.fit_transform(sentiment_scores)

# 计算类权重
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_y_onehot), y=train_y_onehot)
weights = {i : class_weights[i] for i in range(len(class_weights))}

# 将类权重添加到逻辑回归模型
logistic_model = LogisticRegression(class_weight=weights)

# 可能选择更多特征
selector = RFE(logistic_model, n_features_to_select=5, step=1)
selector.fit(train_x_onehot, train_y) ##TO CHANGE!
# grid_search.fit(test_x_onehot, test_y)

# 定义要搜索的参数网格
param_grid = {
    'C': np.logspace(-4, 4, 10),
    'penalty': ['l2'],
    'solver': ['liblinear', 'lbfgs']
}

# 设置GridSearchCV
grid_search = GridSearchCV(logistic_model, param_grid, cv=3, scoring='accuracy', verbose=3)
grid_search.fit(train_x_onehot, train_y) ##TO CHANGE!
# grid_search.fit(test_x_onehot, test_y)

# 使用最佳参数的模型
best_model = grid_search.best_estimator_

# 预测
test_pred = best_model.predict(test_x_onehot)

Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV 1/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=0.993 total time=   0.3s
[CV 2/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=0.993 total time=   0.4s
[CV 3/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=0.993 total time=   0.4s
[CV 1/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9;, score=0.993 total time=   0.5s
[CV 2/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9;, score=0.993 total time=   0.5s
[CV 3/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9;, score=0.993 total time=   0.6s
[CV 1/3] END colsample_bytree=0.3, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0;, score=0.993 total time=   0.5s
[CV 2/

/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/3] END C=21.54434690031882, penalty=l2, solver=lbfgs;, score=0.989 total time=   4.8s


/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/3] END C=21.54434690031882, penalty=l2, solver=lbfgs;, score=0.992 total time=   4.2s


...

[CV 1/3] END C=10000.0, penalty=l2, solver=lbfgs;, score=0.988 total time=   5.4s


/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/3] END C=10000.0, penalty=l2, solver=lbfgs;, score=0.989 total time=   4.2s


/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/3] END C=10000.0, penalty=l2, solver=lbfgs;, score=0.990 total time=   4.6s


In [105]:
# 评估
print("Classification Report:\n", classification_report(test_y, test_pred))
conf_matrix = confusion_matrix(test_y, test_pred)
print("Confusion Matrix:\n", conf_matrix)

# 评估：ndcg
print("NDCG@1:\n", ndcg_score([test_y], [test_pred], k=1))
print("NDCG@5:\n", ndcg_score([test_y], [test_pred], k=5))
print("NDCG@10:\n", ndcg_score([test_y], [test_pred], k=10))
print("NDCG:\n", ndcg_score([test_y], [test_pred]))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6354
         1.0       0.00      0.00      0.00        46

    accuracy                           0.99      6400
   macro avg       0.50      0.50      0.50      6400
weighted avg       0.99      0.99      0.99      6400

Confusion Matrix:
 [[6354    0]
 [  46    0]]
NDCG@1:
 0.0071875
NDCG@5:
 0.007187499999999999
NDCG@10:
 0.0071875
NDCG:
 0.3458997499253763


/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wuziqing/.local/share/virtualenvs/MTL_project-BiIXOKA8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

Feature importance

In [107]:
selector

RFE(estimator=LogisticRegression(class_weight={0: 1.0}), n_features_to_select=5)

In [106]:
feature_importance = list(zip(test_x_onehot.columns, selector.ranking_))
feature_importance.sort(key=lambda x: x[1])
print("Feature importances:\n", feature_importance)

AttributeError: 'RFE' object has no attribute 'ranking_'

Without class weights

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 对 dominant_topic 进行独热编码


# 标准化
scaler = StandardScaler()
sentiment_scores = train_x_onehot['sentiment_score'].values.reshape(-1, 1)
train_x_onehot['sentiment_score'] = scaler.fit_transform(sentiment_scores)

# 将类权重添加到逻辑回归模型
logistic_model = LogisticRegression()

# 可能选择更多特征
selector = RFE(logistic_model, n_features_to_select=5, step=1)
# selector.fit(train_x_onehot, train_y) ##TO CHANGE!
grid_search.fit(test_x_onehot, test_y)

# 定义要搜索的参数网格
param_grid = {
    'C': np.logspace(-4, 4, 10),
    'penalty': ['l2'],
    'solver': ['liblinear', 'lbfgs']
}

# 设置GridSearchCV
grid_search = GridSearchCV(logistic_model, param_grid, cv=3, scoring='accuracy', verbose=3)
# grid_search.fit(train_x_onehot, train_y) ##TO CHANGE!
grid_search.fit(test_x_onehot, test_y)

# 使用最佳参数的模型
best_model = grid_search.best_estimator_

# 预测
test_pred = best_model.predict(test_x_onehot)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END C=9.999999999999999e-05, penalty=l2, solver=liblinear;, score=0.952 total time=  17.6s
[CV 2/3] END C=9.999999999999999e-05, penalty=l2, solver=liblinear;, score=0.952 total time=  18.5s
[CV 3/3] END C=9.999999999999999e-05, penalty=l2, solver=liblinear;, score=0.952 total time=  17.4s
[CV 1/3] END C=9.999999999999999e-05, penalty=l2, solver=lbfgs;, score=0.952 total time=  30.7s
[CV 2/3] END C=9.999999999999999e-05, penalty=l2, solver=lbfgs;, score=0.952 total time=  51.9s
[CV 3/3] END C=9.999999999999999e-05, penalty=l2, solver=lbfgs;, score=0.952 total time= 1.0min
[CV 1/3] END C=0.000774263682681127, penalty=l2, solver=liblinear;, score=0.952 total time=  18.5s
[CV 2/3] END C=0.000774263682681127, penalty=l2, solver=liblinear;, score=0.952 total time=  18.6s
[CV 3/3] END C=0.000774263682681127, penalty=l2, solver=liblinear;, score=0.952 total time=  18.6s
[CV 1/3] END C=0.000774263682681127, penalty=l2, solve

In [ ]:
# 评估
print("Classification Report:\n", classification_report(test_y, test_pred))
conf_matrix = confusion_matrix(test_y, test_pred)
print("Confusion Matrix:\n", conf_matrix)

# 评估：ndcg
print("NDCG@1:\n", ndcg_score([test_y], [test_pred], k=1))
print("NDCG@5:\n", ndcg_score([test_y], [test_pred], k=5))
print("NDCG@10:\n", ndcg_score([test_y], [test_pred], k=10))
print("NDCG:\n", ndcg_score([test_y], [test_pred]))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      1.00      0.98   1886055
         1.0       0.55      0.04      0.07     95054

    accuracy                           0.95   1981109
   macro avg       0.75      0.52      0.52   1981109
weighted avg       0.93      0.95      0.93   1981109

Confusion Matrix:
 [[1883050    3005]
 [  91393    3661]]


Feature importance

In [ ]:
feature_importance = list(zip(test_x_onehot.columns, selector.ranking_))
feature_importance.sort(key=lambda x: x[1])
print("Feature importances:\n", feature_importance)

Feature importances:
 [('eastmoney_robo_journalism', 1), ('SMA_robo_journalism', 1), ('colon_mark', 1), ('topic_3', 1), ('topic_4', 1), ('topic_0', 2), ('media_robo_journalism', 3), ('topic_1', 4), ('question_mark', 5), ('topic_2', 6), ('exclamation_mark', 7), ('sentiment_score', 8), ('month', 9), ('article_source_cate', 10), ('article_author', 11)]
